In [167]:
# pip install requests beautifulsoup4 pandas

In [258]:
# import requests
# from bs4 import BeautifulSoup
# import pandas as pd

In [260]:
# url = 'https://startups.startupmission.in/startups/8qjEZ'
# response = requests.get(url)


In [262]:
# soup = BeautifulSoup(response.content, 'html.parser')


In [264]:
# elements = soup.find_all('div', class_="flex gap-1 items-start")
# product_names = soup.find_all('div', class_ = 'text-xl font-bold mb-3')
# product = product_names[1].text.strip()

In [266]:
# print(elements[0].text.strip() if len(elements) > 0 else "Not Found")

In [268]:
# print(elements[3].text.strip() if len(elements) > 0 else "Not Found")

In [270]:
# for i in range(len(elements)):
#     print(elements[i].text.strip() if len(elements) > 0 else "Not Found")

In [272]:

# sector = " ".join(elements[0].text.split()) if len(elements) > 0 else "Not Found"

# industry = " ".join(elements[1].text.split()) if len(elements) > 0 else "Not Found"


# bizmodel = " ".join(elements[2].text.split()) if len(elements) > 0 else "Not Found"

# tech = " ".join(elements[3].text.split()) if len(elements) > 0 else "Not Found"

# print(sector)

In [274]:
# data = {
#     'product': [product],
#     'sector': [sector],
#     'Industry': [industry],
#     'Business Models': [bizmodel],
#     'Technology': [tech]
# }

In [276]:
# df = pd.DataFrame(data)
# df
# # Print DataFrame
# # print(df)

In [338]:
import requests
from bs4 import BeautifulSoup
import json
import re

def extract_company_data(url):
    """
    Returns:
        Dictionary containing company information
    """
    try:
        # Sending GET request to the URL
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        # Parse HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Initialize company data dictionary
        company_data = {
            'company_name': '',
            'website': '',
            'email': '',
            'linkedin': '',
            'incorporation_date': '',
            'incorporation_type': '',
            'registered_address': '',
            'office_address': '',
            'sectors': [],
            'business_models': [],
            'industries': [],
            'technologies': [],
            'team': [],
            'products': []
        }
        
        # Extract company name
        company_name_elem = soup.select_one('.md\\:mt-10.mt-2.font-bold.text-xl.text-blue-700.mb-5')
        if company_name_elem:
            company_data['company_name'] = company_name_elem.text.strip()
        
        # Extract website URL
        website_link = soup.select_one('a[href^="http"]:has(svg.icon-tabler-world-www)')
        if website_link:
            company_data['website'] = website_link.get('href')
        
        # Extract LinkedIn URL
        linkedin_link = soup.select_one('a[href*="linkedin.com"]:has(svg.icon-tabler-brand-linkedin)')
        if linkedin_link:
            company_data['linkedin'] = linkedin_link.get('href')
        
        # Extract basic company information
        info_sections = soup.select('.bg-white.shadow-md.rounded-lg.mt-5.px-5.py-5.space-y-3')
        for section in info_sections:
            # Extract incorporation information
            incorporation_date = section.select_one('div:-soup-contains("Incorporation Date") + div')
            if incorporation_date:
                company_data['incorporation_date'] = incorporation_date.text.strip()
            
            incorporation_type = section.select_one('div:-soup-contains("Incorporation Type") + div')
            if incorporation_type:
                company_data['incorporation_type'] = incorporation_type.text.strip()
            
            # Extract address information
            registered_address = section.select_one('div:-soup-contains("Registered Address") + div')
            if registered_address:
                company_data['registered_address'] = registered_address.text.strip()
            
            office_address = section.select_one('div:-soup-contains("Office Address") + div')
            if office_address:
                company_data['office_address'] = office_address.text.strip()
            
            # Extract team information
            team_members = section.select('.flex.flex-col.items-center.justify-start.text-center')
            for member in team_members:
                name = member.select_one('.font-bold')
                role = name.find_next_sibling() if name else None
                
                if name and role:
                    company_data['team'].append({
                        'name': name.text.strip(),
                        'role': role.text.strip()
                    })
        
        # Extracting sectors, business models, industries, technologies
        info_grid = soup.select_one('.grid.md\\:grid-cols-2.gap-2')
        if info_grid:
            # Extract sectors
            sectors = info_grid.select_one('.flex.gap-1.items-start:-soup-contains("Sector:") .flex.flex-wrap.gap-1.items-start span')
            if sectors:
                company_data['sectors'] = [sector.text.strip() for sector in info_grid.select('.flex.gap-1.items-start:-soup-contains("Sector:") .flex.flex-wrap.gap-1.items-start span')]
            
            # Extract business models
            business_models = info_grid.select_one('.flex.gap-1.items-start:-soup-contains("Business Model:") .flex.flex-wrap.gap-1.items-start span')
            if business_models:
                company_data['business_models'] = [model.text.strip() for model in info_grid.select('.flex.gap-1.items-start:-soup-contains("Business Model:") .flex.flex-wrap.gap-1.items-start span')]
            
            # Extract industries
            industries = info_grid.select_one('.flex.gap-1.items-start:-soup-contains("Industry:") .flex.flex-wrap.gap-1.items-start span')
            if industries:
                company_data['industries'] = [industry.text.strip() for industry in info_grid.select('.flex.gap-1.items-start:-soup-contains("Industry:") .flex.flex-wrap.gap-1.items-start span')]
            
            # Extract technologies
            technologies = info_grid.select_one('.flex.gap-1.items-start:-soup-contains("Technology:") .flex.flex-wrap.gap-1.items-start span')
            if technologies:
                company_data['technologies'] = [tech.text.strip() for tech in info_grid.select('.flex.gap-1.items-start:-soup-contains("Technology:") .flex.flex-wrap.gap-1.items-start span')]
        
        # Extract product information
        product_sections = soup.select('#products .bg-white.shadow-md.rounded-lg.mt-5.px-5.py-5')
        for product in product_sections:
            product_name = product.select_one('.text-xl.font-bold.mb-3')
            product_desc = product.select_one('.text-justify')
            
            if product_name:
                product_data = {
                    'name': product_name.text.strip(),
                    'description': product_desc.text.strip() if product_desc else '',
                    'sectors': [],
                    'industries': [],
                    'business_models': [],
                    'technologies': []
                }
                
                # Extract product-specific sectors
                product_sectors = product.select('.flex.gap-1.items-start:-soup-contains("Sector:") .flex.flex-wrap.gap-1.items-start span')
                if product_sectors:
                    product_data['sectors'] = [sector.text.strip() for sector in product_sectors]
                
                # Extract product-specific industries
                product_industries = product.select('.flex.gap-1.items-start:-soup-contains("Industry:") .flex.flex-wrap.gap-1.items-start span')
                if product_industries:
                    product_data['industries'] = [industry.text.strip() for industry in product_industries]
                
                # Extract product-specific business models
                product_models = product.select('.flex.gap-1.items-start:-soup-contains("Business Models:") .flex.flex-wrap.gap-1.items-start span')
                if product_models:
                    product_data['business_models'] = [model.text.strip() for model in product_models]
                
                # Extract product-specific technologies
                product_techs = product.select('.flex.gap-1.items-start:-soup-contains("Technology:") .flex.flex-wrap.gap-1.items-start span')
                if product_techs:
                    product_data['technologies'] = [tech.text.strip() for tech in product_techs]
                
                company_data['products'].append(product_data)
        
        return company_data
    
    except Exception as e:
        print(f"Error extracting data: {e}")
        return None

# def save_to_json(company_data, output_file='company_data.json'):
#     """Save extracted company data to a JSON file"""
#     try:
#         with open(output_file, 'w', encoding='utf-8') as f:
#             json.dump(company_data, f, indent=4, ensure_ascii=False)
#         print(f"Data saved to {output_file}")
#     except Exception as e:
#         print(f"Error saving data: {e}")

# def print_company_summary(company_data):
#     """Print a summary of the company data"""
#     if not company_data:
#         print("No data available")
#         return

# For console output
    
    # print("\n" + "="*50)
    # print(f"COMPANY: {company_data['company_name']}")
    # print("="*50)
    
    # print(f"\nWebsite: {company_data['website']}")
    # print(f"LinkedIn: {company_data['linkedin']}")
    # print(f"Incorporation: {company_data['incorporation_date']} ({company_data['incorporation_type']})")
    
    # print("\nSECTORS:")
    # for sector in company_data['sectors']:
    #     print(f"- {sector}")
    
    # print("\nBUSINESS MODELS:")
    # for model in company_data['business_models']:
    #     print(f"- {model}")
    
    # print("\nINDUSTRIES:")
    # for industry in company_data['industries']:
    #     print(f"- {industry}")
    
    # print("\nTECHNOLOGIES:")
    # for tech in company_data['technologies']:
    #     print(f"- {tech}")
    
    # print("\nADDRESSES:")
    # print(f"Registered: {company_data['registered_address']}")
    # print(f"Office: {company_data['office_address']}")
    
    # print("\nTEAM MEMBERS:")
    # for member in company_data['team']:
    #     print(f"- {member['name']} ({member['role']})")
    
    # print("\nPRODUCTS:")
    # for i, product in enumerate(company_data['products'], 1):
    #     print(f"\n{i}. {product['name']}")
    #     print(f"   Description: {product['description']}")
        
    #     if product['sectors']:
    #         print("   Sectors: " + ", ".join(product['sectors']))
        
    #     if product['industries']:
    #         print("   Industries: " + ", ".join(product['industries']))
        
    #     if product['business_models']:
    #         print("   Business Models: " + ", ".join(product['business_models']))
        
    #     if product['technologies']:
    #         print("   Technologies: " + ", ".join(product['technologies']))
# Main Function

if __name__ == "__main__":
    # List of URLs to scrape 
    print("URLs for Ernakulam or Thiruvananthapuram:")
    # leng = 0  
    urls_to_scrape = []
    locationDetails = []
    inUrls = [
        'https://startups.startupmission.in/startups?sector=Banking%2C+Financial+Services+and+Insurance&industry=FinTech',
        'https://startups.startupmission.in/startups?sector=Banking%2C+Financial+Services+and+Insurance&industry=FinTech&page=2',
        'https://startups.startupmission.in/startups?sector=Banking%2C+Financial+Services+and+Insurance&industry=FinTech&page=3',
        'https://startups.startupmission.in/startups?sector=Banking%2C+Financial+Services+and+Insurance&industry=FinTech&page=4',
        'https://startups.startupmission.in/startups?sector=Banking%2C+Financial+Services+and+Insurance&industry=FinTech&page=5'
    ]

    # Get the page
    for inUrl in inUrls:
        response = requests.get(inUrl)
        html_code = response.text

        # Parse the HTML
        soup = BeautifulSoup(html_code, 'html.parser')

        # Initialize a list to store the URLs
        urls = []

    # Find all startup cards (the <a> elements)
        startup_cards = soup.select('.container .grid a')

    # Alternative if the above doesn't work (the HTML might be incomplete)
        if not startup_cards:
            startup_cards = soup.find_all('a')

# Loop through each card
        for card in startup_cards:
        # Find the location text
            location_element = card.select_one('.text-sm')
            if location_element:
                location_text = location_element.text.strip().lower()
            # Check if the location contains "ernakulam" or "thiruvananthapuram"
                if "ernakulam" in location_text or "thiruvananthapuram" in location_text:
                    # Add the URL to our list
                    urls.append(card['href'])
                    locationDetails.append(location_text)

# Print the results
        for url in urls:
            urls_to_scrape.append(url)
        # leng = leng + len(urls)
    # print(urls_to_scrape)
    # print(leng)
    # print(len(locationDetails))
# These are individually given urls in version 1
#     urls_to_scrape = [
#         # "https://startups.startupmission.in/startups/pQlq0",
#         # "https://startups.startupmission.in/startups/8Vg5Z",
#         # "https://startups.startupmission.in/startups/py658",
#         # "https://startups.startupmission.in/startups/8XkLY",
#         # "https://startups.startupmission.in/startups/ZD7x2",
#         # "https://startups.startupmission.in/startups/Z3eAx",
#         # "https://startups.startupmission.in/startups/pzgXQ",
#         # "https://startups.startupmission.in/startups/pkkJm",
#         # "https://startups.startupmission.in/startups/Z3ezJ",
#         # "https://startups.startupmission.in/startups/p69E0",
#         # "https://startups.startupmission.in/startups/8XkRD",
#         # "https://startups.startupmission.in/startups/0Brd0",
#         # "https://startups.startupmission.in/startups/Z1ERZ",
#         # "https://startups.startupmission.in/startups/8n1eD",
#         # "https://startups.startupmission.in/startups/pQXX0",
#         # "https://startups.startupmission.in/startups/8XaDZ",
#         # "https://startups.startupmission.in/startups/ZNrRZ",
#         # "https://startups.startupmission.in/startups/Z9dQ8",
#         # "https://startups.startupmission.in/startups/8LWxp",
#         # "https://startups.startupmission.in/startups/04Nn8",
#         # "https://startups.startupmission.in/startups/ZaPn8",
#         # "https://startups.startupmission.in/startups/ZEebk",
#         # "https://startups.startupmission.in/startups/Z9WMP",
#         # "https://startups.startupmission.in/startups/8AvlZ",
#         # "https://startups.startupmission.in/startups/pkVr0",
#         # "https://startups.startupmission.in/startups/04q2p",
#         # "https://startups.startupmission.in/startups/8L6r0",
#         # "https://startups.startupmission.in/startups/NZOgZ",
#         # "https://startups.startupmission.in/startups/04Yap",
#         # "https://startups.startupmission.in/startups/poa3Z",
#         # "https://startups.startupmission.in/startups/8xOBb",
#         # "https://startups.startupmission.in/startups/ZEk1p"
#         'https://startups.startupmission.in/startups/8nJa8'
#     ]
    
    # Processing each URL
    all_companies_data = []
    
    for url in urls_to_scrape:
        print(f"Processing: {url}")
        
        # Extracting data
        company_data = extract_company_data(url)
        
        # Printing summary
        # print_company_summary(company_data)
        
        # Save individual JSON file to check
        # json_filename = f"{company_data['company_name'].replace(' ', '_').lower()}_data.json"
        # save_to_json(company_data, json_filename)
        
        # Add to the list
        all_companies_data.append(company_data)
    
    # Convert list of dictionaries to DataFrame
    df = pd.DataFrame(all_companies_data)
    
    # Processing columns containing lists or dictionaries
    for col in df.columns:
        if len(df) > 0 and df[col].notna().any():
            sample_value = df[col].iloc[0]
            
            # Handle list of strings
            if isinstance(sample_value, list) and all(isinstance(item, str) for item in sample_value):
                df[col] = df[col].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
            
            # Handle list of dictionaries - convert to JSON string
            elif isinstance(sample_value, list) and any(isinstance(item, dict) for item in sample_value):
                df[col] = df[col].apply(lambda x: json.dumps(x) if isinstance(x, list) else x)
    
    # Create a separate DataFrame for products
    product_rows = []
    for company in all_companies_data:
        company_name = company.get('company_name', '')
        products = company.get('products', [])
        
        if products:
            for product in products:
                if isinstance(product, dict):
                    product_dict = {
                        'company_name': company_name,
                        'product_name': product.get('name', ''),
                        'product_description': product.get('description', '')
                    }
                    
                    # Handle nested lists within products
                    for key in ['sectors', 'industries', 'business_models', 'technologies']:
                        if key in product:
                            values = product.get(key, [])
                            product_dict[f'product_{key}'] = ', '.join(values) if isinstance(values, list) else values
                    
                    product_rows.append(product_dict)
    
    products_df = pd.DataFrame(product_rows) if product_rows else None
    df['Location'] = locationDetails
    
    # Save to Excel file with multiple sheets
    excel_filename = "companies_data_check2.xlsx"
    with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name='Companies', index=False)
        
        if products_df is not None and not products_df.empty:
            products_df.to_excel(writer, sheet_name='Products', index=False)
    
    print(f"Processed {len(all_companies_data)} companies successfully.")
    print(f"Data saved to {excel_filename}")
    # print(len(locationDetails))

URLs for Ernakulam or Thiruvananthapuram:
Processing: https://startups.startupmission.in/startups/ZdX4Z
Processing: https://startups.startupmission.in/startups/p6rgE
Processing: https://startups.startupmission.in/startups/poXQV
Processing: https://startups.startupmission.in/startups/poE3Z
Processing: https://startups.startupmission.in/startups/8Xoyp
Processing: https://startups.startupmission.in/startups/ZDVEp
Processing: https://startups.startupmission.in/startups/0JgYp
Processing: https://startups.startupmission.in/startups/8qjEZ
Processing: https://startups.startupmission.in/startups/07mQA
Processing: https://startups.startupmission.in/startups/ZEw1p
Processing: https://startups.startupmission.in/startups/0wgm1
Processing: https://startups.startupmission.in/startups/pQynE
Processing: https://startups.startupmission.in/startups/Z1M5Z
Processing: https://startups.startupmission.in/startups/pvVql
Processing: https://startups.startupmission.in/startups/8VDAv
Processing: https://startups

In [ ]:
# from bs4 import BeautifulSoup
# import requests

# print("URLs for Ernakulam or Thiruvananthapuram:")
# # leng = 0  
# urls_to_scrape = []
# inUrls = [
#     'https://startups.startupmission.in/startups?sector=Banking%2C+Financial+Services+and+Insurance&industry=FinTech',
#     'https://startups.startupmission.in/startups?sector=Banking%2C+Financial+Services+and+Insurance&industry=FinTech&page=2',
#     'https://startups.startupmission.in/startups?sector=Banking%2C+Financial+Services+and+Insurance&industry=FinTech&page=3',
#     'https://startups.startupmission.in/startups?sector=Banking%2C+Financial+Services+and+Insurance&industry=FinTech&page=4',
#     'https://startups.startupmission.in/startups?sector=Banking%2C+Financial+Services+and+Insurance&industry=FinTech&page=5'
# ]

# # Get the page
# for inUrl in inUrls:
#     response = requests.get(inUrl)
#     html_code = response.text

#     # Parse the HTML
#     soup = BeautifulSoup(html_code, 'html.parser')

#     # Initialize a list to store the URLs
#     urls = []

# # Find all startup cards (the <a> elements)
#     startup_cards = soup.select('.container .grid a')

#     # Alternative if the above doesn't work (the HTML might be incomplete)
#     if not startup_cards:
#         startup_cards = soup.find_all('a')

# # Loop through each card
#     for card in startup_cards:
#         # Find the location text
#         location_element = card.select_one('.text-sm')
#         if location_element:
#             location_text = location_element.text.strip().lower()
        
#         # Check if the location contains "ernakulam" or "thiruvananthapuram"
#             if "ernakulam" in location_text or "thiruvananthapuram" in location_text:
#                 # Add the URL to our list
#                 urls.append(card['href'])

# # Print the results
      
#     for url in urls:
#         urls_to_scrape.append(url)
#     leng = leng + len(urls)
# urls_to_scrape
# # print(leng)